https://nbviewer.jupyter.org/github/AdamSwenson/CanvasHacks/blob/master/Notebooks/Credit%20only%20assignment%20tools.ipynb?flush_cache=true

# Get started

Click the icon with 3 rings in the upper right (the tooltip says 'Execute on Binder') to create your own mini-server to run this notebook.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also press shift+return to run it.

Try it with the next cell:

In [ ]:
print("You've successfully run this cell!")


You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( [ * ]). Some tasks may take a couple of minutes to run, so it may appear frozen.

The next 2 cells will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/AdamSwenson/CanvasHacks)


In [ ]:
%cd ../
from canvasapi import Canvas
from CanvasHacks import environment
from CanvasHacks.Definitions.generic import GenericCreditNoCredit
from CanvasHacks.Api.UploadGradeTools import make_upload_button
from CanvasHacks.Repositories.DataManagement import DataStore, DataStoreNew
from CanvasHacks.Repositories.submissions import SubmissionRepository
from CanvasHacks.Repositories.students import StudentRepository
from CanvasHacks.GradingHandlers.journal import JournalGrader
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.CredentialFields import make_credentials_input
make_credentials_input()

WARNING: THIS TEMPORARILY ONLY WORKS FOR 1 CLASS ID. 
IF YOU NEED TO GRADE ASSIGNMENTS FOR MORE THAN ONE CLASS, AFTER GRADING ASSIGNMENTS FOR THE FIRST CLASS, RETURN TO THIS POINT, CLICK 'RESET', ADD THE NEXT CLASS ID, AND CONTINUE ON.

## Canvas url

Finally, we need the url of your campus canvas installation. When we make requests to the canvas server, we will be using this in our requests. For details, see the Canvas API documentation: https://canvas.instructure.com/doc/api/index.html

For CSUN users, you do not need to touch this. The value in the box should be: 
    
    https://canvas.csun.edu/api/v1/courses

For non-csun users, make sure you provide the equivalent up to the '/courses'

Just paste the url in the box, it will be read when needed

Do not add a slash at the end after '/courses'! 

In [ ]:
make_canvas_url_input(value="https://canvas.csun.edu/api/v1/courses")

#  Figure out what needs grading¶

Run the cell below. It will display buttons for each ungraded assignment where at least one student has submitted the assignment. 

Use these buttons to select which assignments to grade. Unselected assignments are blue; selected are green.

NOTE: THIS WILL SHOW ALL UNGRADED ACTIVITIES. THESE TOOLS WILL NOT WORK ON ALL OF THEM. 
AS PRESENTLY CONFIGURED, YOU CAN ONLY GRADE ASSIGNMENTS (NOT QUIZ-TYPE ACTIVITIES) THAT USE A TEXT-ENTRY BOX OR UPLOADED FILES. 

In [ ]:
make_assignment_chooser()

#  Download and process student submissions

In [ ]:
environment.CONFIG.course

In [ ]:
# not needed w file based config
environment.CONFIG.initialize_canvas_objs()

results = []

for a in environment.CONFIG.assignments:
    # canvas api object
    assignment = environment.CONFIG.course.get_assignment(int(a[0]))
    # activity object to define the features 
    assignment_defin = GenericCreditNoCredit(**assignment.attributes)
    # Download submissions
    subRepo = SubmissionRepository(assignment)
    # parse out already graded submissions
    subRepo.data =[j for j in subRepo.data if j.grade != 'complete']
    # shove the activity onto a sub repo so it will resemble
    # a quizrepo for the grader
    subRepo.activity = assignment_defin
    # Initialize the package for results
    store = DataStoreNew(assignment_defin)
    # provisionally determine credit
    grader = JournalGrader(subRepo)
    store.results = grader.grade()
    results.append(store)

# Load student names etc
studentRepo = StudentRepository(environment.CONFIG.course)
studentRepo.download()

# Read student work and check that properly categorized

In [ ]:
for s in results:
    make_assignment_header(s)
    make_consolidated_text_fields(s, studentRepo)

# Upload grades to canvas

If you're just experimenting and don't want to upload grades, stop here.

Clicking the buttons which appear after running the next cell will upload grades to canvas. That can't be undone.

In [ ]:
for store in results:
    make_upload_button(store)